In [26]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

from config import ACCESS_KEY, SECRET_KEY, TOKEN

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.strptime('2024-09-01 00:00:00','%Y-%m-%d %H:%M:%S')
    #now = datetime.datetime.now(pytz.timezone(time_zone))
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m')
            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
last_month = get_now_datetime_str()['last_month']

limit = 100
offset = 1

now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']


token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [27]:
print(now)
print(now_date)
print(yesterday)
print(yesterday_data)
print(start_of_month_data)

print(day)
print(month)
print(year)

2024-09-01 00:00:00
2024-09-01
2024-08-31 00:00:00
2024-08-31
2024-08-01
31
08
2024


In [28]:
FOLDER = 'all_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [29]:
FOLDER = 'revenue_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [30]:
FOLDER = 'all_flats_rep_partners'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [31]:
FOLDER = 'conected_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [32]:
FOLDER = 'address_flats_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [33]:
FOLDER = 'count_of_online_intercoms_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [34]:
FOLDER = 'count_of_instaletion_point_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [35]:
FOLDER = 'arhive_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [36]:
FOLDER = 'all_partners_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [37]:
FOLDER = 'arhive_enterprise_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [38]:
FOLDER = 'enterprice_commerce_intercom_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [39]:
FOLDER = 'accruals_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [40]:
FOLDER = 'count_of_blocks_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [41]:
FOLDER = 'count_of_io_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [42]:
FOLDER = 'partner_programm_amount_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [43]:
FOLDER = 'other_withdrawals_amount_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [44]:
FOLDER =  'revenue_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False)

In [45]:
FOLDER =  'number_of_services_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [46]:
FOLDER =  'accruals_commercial_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [47]:
FOLDER =  'partners_count_enterprice_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [48]:
FOLDER =  'revenue_enterpise_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [49]:
FOLDER =  'number_of_services_enterprise_rep_partner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [50]:
FOLDER =  'accruals_enterprise_rep_pertner'
temp_file_pd = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(last_month)}.csv',sep=",")
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv',sep=",", index=False) 

In [51]:
FOLDER = 'partners_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data,year,month)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [52]:
FOLDER = 'separate_cameras_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [53]:
FOLDER = 'intercom_uuid_count_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [54]:
FOLDER = 'number_of_services_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [124]:
FOLDER = 'yandex_payments_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [207]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.strptime('2024-09-01 00:00:00','%Y-%m-%d %H:%M:%S')
    #now = datetime.datetime.now(pytz.timezone(time_zone))
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m')

            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
last_month = get_now_datetime_str()['last_month']

limit = 100
offset = 1

now = get_now_datetime_str()['now'] 
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']

In [208]:
print(now)
print(now_date)
print(yesterday)
print(yesterday_data)
print(start_of_month_data)

print(day)
print(month)
print(year)

2024-09-01 00:00:00
2024-09-01
2024-08-31 00:00:00
2024-08-31
2024-08-01
31
08
2024


In [209]:
FOLDER = 'expense_greensms_sum_rep_partner'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r', encoding='utf-8').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/report_partner/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

___

In [210]:
import base64
yandex_payments_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/yandex_payments_rep_partner/{int(year)}/{int(month)}.csv')
yandex_payments_rep_partner['service_name'] = yandex_payments_rep_partner['service_name'].apply(lambda x: base64.b64decode(x).decode('utf-8'))
yandex_payments_rep_partner.index = yandex_payments_rep_partner.service_name
yandex_payments_rep_partner = yandex_payments_rep_partner.drop(['service_name'],axis=1)
yandex_payments_rep_partner = yandex_payments_rep_partner.transpose()
yandex_payments_rep_partner = yandex_payments_rep_partner.reset_index().drop(['index'],axis=1)
yandex_payments_rep_partner = yandex_payments_rep_partner.rename(columns={'Managed Service for Redis™':'Managed Service for Redis'})
yandex_payments_rep_partner

service_name,Container Registry,Yandex SpeechKit,Compute Cloud,Lockbox Service,Managed Service for Kubernetes,Yandex Cloud Logging,Cloud Functions,Managed Service for PostgreSQL,Load Balancer,Managed Service for YDB,Object Storage,Yandex Query,Managed Service for Redis,Cloud DNS,Key Management Service,Managed Service for Clickhouse,VPC
0,210.494678,7583.53836,363621.1192,279.33143,53713.93804,0.0,0.0,88380.87228,12372.14319,35.098776,38884.55331,24089.69531,15300.77819,17616.50161,728.997663,48758.97389,19110.41888


In [211]:
partners_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partners_count_rep_partner/{int(year)}/{int(month)}.csv')
partners_count_rep_partner

,year,month,report_date,number_of_clients,count_start,count_PRO,count_sleaping,count_only_warehouse,connected_0_growth_intercom_0_sleaping,connected_0_growth_intercom_0_only_warehouse,connected_10_growth_intercom_less_0,connected_10_growth_intercom_greater_0,connected_100_growth_intercom_less_0,connected_100_growth_intercom_greater_0,connected_1000_growth_intercom_less_0,connected_1000_growth_intercom_greater_0,connected_greater_1000_growth_intercom_less_0,connected_greater_1000_growth_intercom_greater_0
0,2024,8,'2024-08-31',824,684,140,311,322,311,11,274,0,177,0,49,0,2,0


In [212]:
address_flats_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/address_flats_rep_partner/{int(year)}/{int(month)}.csv')
address_flats_rep_partner

,sum_flat_uuid,count_houses_uuid,sum_flat_uuid_PRO,sum_flat_uuid_Start,sum_address_uuid,sum_address_uuid_count_PRO,sum_address_uuid_count_Start,avg_flat_uuid,avg_flat_uuid_PRO,avg_flat_uuid_Start,avg_address_uuid,avg_address_uuid_count_PRO,avg_address_uuid_count_Start
0,727036,7229,504182,222854,24289,16555,7734,1036,3601,397,35,118,14


In [213]:
all_partners_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_partners_rep_partner/{int(year)}/{int(month)}.csv')
all_partners_rep_partner

,all_partners_rep_partner
0,8161


In [214]:
arhive_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/arhive_rep_partner/{int(year)}/{int(month)}.csv')
arhive_rep_partner

,cameras_with_archive_3,cameras_with_archive_5,cameras_with_archive_7,cameras_with_archive_14,cameras_with_archive_30,cameras_with_archive_3_pro,cameras_with_archive_5_pro,cameras_with_archive_7_pro,cameras_with_archive_14_pro,cameras_with_archive_30_pro,cameras_with_archive_3_start,cameras_with_archive_5_start,cameras_with_archive_7_start,cameras_with_archive_14_start,cameras_with_archive_30_start
0,6717,778,1029,5,23,5381,553,444,5,23,1336,225,585,0,0


In [215]:
conected_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/conected_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
conected_intercoms_rep_partner

,intercom_uuid_start_pro_count,count_intercoms_PRO,count_intercoms_Start,count_connected_intercoms_PRO,count_connected_intercoms_Start,count_connected_intercoms,disconnected_intercoms_30,disconnected_intercoms_60,disconnected_intercoms_90
0,35172,22036,13136,16975,8007,24982,8762,8142,7614


In [216]:
count_of_instaletion_point_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_instaletion_point_rep_partner/{int(year)}/{int(month)}.csv')
count_of_instaletion_point_rep_partner

,count_of_instaletion_point,count_of_instaletion_point_start,count_of_instaletion_point_pro,count_of_instaletion_point_enterprise
0,25934,8350,16722,862


In [217]:
separate_cameras_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/separate_cameras_rep_partner/{int(year)}/{int(month)}.csv')
separate_cameras_rep_partner

,intercom_with_archive_3_from_partner,intercom_with_archive_5_from_partner,intercom_with_archive_7_from_partner,intercom_with_archive_14_from_partner,intercom_with_archive_30_from_partner,cameras_with_archive_3_from_partner,cameras_with_archive_5_from_partner,cameras_with_archive_7_from_partner,cameras_with_archive_14_from_partner,cameras_with_arcive_30h_from_partner,...,all_cameras_sputnic_without_intercom,instaleted_cameras_sputnic_without_intercom,all_camera_sputnic_without_intercom_start,all_camera_sputnic_without_intercom_pro,all_camera_sputnic_without_intercom_enterprise,foreign_camera_without_intercom,instaleted_cameras_foreign_without_intercom,all_camera_foreign_without_intercom_start,all_camera_foreign_without_intercom_pro,all_camera_foreign_without_intercom_enterprise
0,10257,639,522,5,0,1518,143,692,0,23,...,5731,3161,368,2345,448,493,285,7,276,2


In [218]:
all_flats_rep_partners = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_flats_rep_partners/{int(year)}/{int(month)}.csv')
all_flats_rep_partners

,all_flats
0,780814


In [219]:
revenue_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_rep_partner/{int(year)}/{int(month)}.csv')
revenue_rep_partner

,revenue,pro,archive_3_pro,archive_5_pro,archive_7_pro,archive_14_pro,archive_30_pro,archive_3,archive_5,archive_7,archive_14,archive_30,developer
0,1479241.8,708820.2,341978,51719.5,7007,1038.5,'',194608,36940,108264,'',157.5,''


In [220]:
all_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
all_intercoms_rep_partner

,all_intercoms
0,56715


In [221]:
intercom_uuid_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/intercom_uuid_count_rep_partner/{int(year)}/{int(month)}.csv')
intercom_uuid_count_rep_partner

,intercom_uuid_count,installation_point_id_connected_count,all_intercoms_start_count,connected_intercoms_start_count,disconnected_intercoms_start_count,installation_point_id_connected_count_start_sum,all_intercoms_pro_count,connected_intercoms_pro_count,disconnected_intercoms_pro_count,installation_point_id_connected_count_pro_sum,all_intercoms_enterprise_count,connected_intercoms_enterprise_count,disconnected_intercoms_enterprise_count,installation_point_id_connected_count_enterprise_sum
0,54159,26425,12547,7732,4802,8429,21825,16898,4872,17011,19787,8375,11407,997


In [222]:
expense_smsc_sum_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/expense_smsc_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_smsc_sum_rep_partner

,expense_smsc_sum,expense_smsc_sum_code,expense_smsc_sum_application,expense_smsc_sum_password,expense_smsc_sum_repair
0,113043.58,103651.516,1122.733,5065.161,1077.54


In [223]:
expense_greensms_sum_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/expense_greensms_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_greensms_sum_rep_partner

,expense_greensms_sum
0,75948


In [224]:
number_of_services_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_commercial_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_commercial_rep_partner

,PRO_count_commercial,archive_count_3_commercial,pro_commercial_avtograd_count,archive_3_pro_commercial_avtograd_count,pro_commercial_actobe_count,archive_count_3_actobe_commerial,count_safe_city_and_security_sector_pro,count_safe_city_and_security_sector_arhive,pro_dominant_count,archive_3_dominant_count
0,3954,2381,2308,2433,0,924,1012,842,873,873


In [225]:
revenue_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_commercial_rep_partner/{int(year)}/{int(month)}.csv')
revenue_commercial_rep_partner

,pro_commercial,archive_3_pro_commercial,pro_actobe_commercial,archive_3_pro_actobe_commercial,pro_commercial_avtograd,archive_3_pro_commercial_avtograd,safe_city_and_security_sector_pro,safe_city_and_security_sector_arhive,pro_dominant,archive_3_dominant
0,315600,215100,0,83205,184640,218970,80960,442050,69840,78570


In [226]:
revenue_enterpise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_enterpise_rep_partner/{int(year)}/{int(month)}.csv')
revenue_enterpise_rep_partner

,revenue_enterprise,АО ЭРТХ (Цифрал Сервис),R1 (АО ЭРТХ),ООО ПКФ ИНСИТ,billings_enterprise
0,592471,187680,271280,113920,19591


In [227]:
ACCESS_KEY = ACCESS_KEY
SECRET_KEY = SECRET_KEY
BUCKET_NAME = "dwh-asgard"
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'C:/Users/Boris/Desktop/report_partner/ready_report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"partner_report/year={int(year)}/{int(month)}.csv"
    )

partners_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partners_count_rep_partner/{int(year)}/{int(month)}.csv')
address_flats_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/address_flats_rep_partner/{int(year)}/{int(month)}.csv')
all_partners_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_partners_rep_partner/{int(year)}/{int(month)}.csv')
arhive_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/arhive_rep_partner/{int(year)}/{int(month)}.csv')
conected_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/conected_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
count_of_instaletion_point_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_instaletion_point_rep_partner/{int(year)}/{int(month)}.csv')
count_of_online_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_online_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
separate_cameras_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/separate_cameras_rep_partner/{int(year)}/{int(month)}.csv')
all_flats_rep_partners = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_flats_rep_partners/{int(year)}/{int(month)}.csv')
revenue_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_rep_partner/{int(year)}/{int(month)}.csv')
all_intercoms_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/all_intercoms_rep_partner/{int(year)}/{int(month)}.csv')
arhive_enterprise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/arhive_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
enterprice_commerce_intercom_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/enterprice_commerce_intercom_count_rep_partner/{int(year)}/{int(month)}.csv')
intercom_uuid_count_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/intercom_uuid_count_rep_partner/{int(year)}/{int(month)}.csv')
partners_count_enterprice_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partners_count_enterprice_rep_partner/{int(year)}/{int(month)}.csv')
revenue_enterpise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_enterpise_rep_partner/{int(year)}/{int(month)}.csv')
revenue_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/revenue_commercial_rep_partner/{int(year)}/{int(month)}.csv')
accruals_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_rep_partner/{int(year)}/{int(month)}.csv')
accruals_enterprise_rep_pertner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_enterprise_rep_pertner/{int(year)}/{int(month)}.csv')
accruals_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/accruals_commercial_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_commercial_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_commercial_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_enterprise_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_enterprise_rep_partner/{int(year)}/{int(month)}.csv')
number_of_services_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/number_of_services_rep_partner/{int(year)}/{int(month)}.csv')
count_of_blocks_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_blocks_rep_partner/{int(year)}/{int(month)}.csv')
count_of_io_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/count_of_io_rep_partner/{int(year)}/{int(month)}.csv')
partner_programm_amount_rep_partner  = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/partner_programm_amount_rep_partner/{int(year)}/{int(month)}.csv')
other_withdrawals_amount_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/other_withdrawals_amount_rep_partner/{int(year)}/{int(month)}.csv')
expense_smsc_sum_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/expense_smsc_sum_rep_partner/{int(year)}/{int(month)}.csv')
expense_greensms_sum_rep_partner = pd.read_csv(f'C:/Users/Boris/Desktop/report_partner/expense_greensms_sum_rep_partner/{int(year)}/{int(month)}.csv')

    
concated = pd.concat([
    partners_count_rep_partner,
    address_flats_rep_partner,
    all_partners_rep_partner,
    arhive_rep_partner,
    conected_intercoms_rep_partner,
    count_of_instaletion_point_rep_partner,
    count_of_online_intercoms_rep_partner,
    separate_cameras_rep_partner,
    all_flats_rep_partners,
    revenue_rep_partner,
    all_intercoms_rep_partner,
    arhive_enterprise_rep_partner,
    enterprice_commerce_intercom_count_rep_partner,
    intercom_uuid_count_rep_partner,
    partners_count_enterprice_rep_partner,
    revenue_enterpise_rep_partner,
    revenue_commercial_rep_partner,
    accruals_rep_partner,
    accruals_enterprise_rep_pertner,
    accruals_commercial_rep_partner,
    number_of_services_commercial_rep_partner,
    number_of_services_enterprise_rep_partner,
    number_of_services_rep_partner,
    count_of_blocks_rep_partner,
    count_of_io_rep_partner,
    partner_programm_amount_rep_partner,
    other_withdrawals_amount_rep_partner,
    expense_smsc_sum_rep_partner,
    expense_greensms_sum_rep_partner,
    yandex_payments_rep_partner
    ]
    , axis=1   
    )

concated.to_csv(f'C:/Users/Boris/Desktop/report_partner/ready_report/{int(year)}/{int(month)}.csv', sep=',', index=False)
    
get_s3_instance()
upload_dump_to_s3()
